# AMT Homework: Inference for Mixed Populations (2024–2025)

Data: `hemodialysismix.csv` and `hemodialysismix.rds` (equivalent content). Outcome of interest: number of occasions with adequate iron stores (`nriron`) out of the number of measurements (`nr`). Covariates: `AGE`, `SEX`.

Notes:
- All missing data (e.g., `NA`) are treated as valid observations and are included. No deletions.
- Analyses follow the course notes (CAMAN VEM→EM, NPMLE gradient check) and the assignment in `task.md`.
- Figures are saved under `figs/`.


In [ ]:
## 0) Setup: packages, options, directories
suppressPackageStartupMessages({
  library(dplyr)
  library(ggplot2)
  library(readr)
  library(forcats)
  library(jsonlite)
})

set.seed(123)

dir.create("figs", showWarnings = FALSE)

theme_set(theme_bw(base_size = 12))

options(dplyr.summarise.inform = FALSE)



In [ ]:
## 1) Read and prepare data (include missing as valid)
# Prefer CSV; RDS contains the same data
raw <- read_csv("hemodialysismix.csv", show_col_types = FALSE)

# Prepare variables; keep all rows (no deletions), allow nr==0
# Treat SEX missing as explicit level "Unknown"; leave AGE missing but keep as is
# Create a missingness indicator for AGE for regression later

dat <- raw %>%
  mutate(
    SEX = factor(SEX, levels = c(1,2), labels = c("Male","Female")),
    SEX = addNA(SEX),  # includes NA as a level
    SEX = fct_explicit_na(SEX, na_level = "Unknown"),
    AGE = suppressWarnings(as.numeric(AGE)),
    age_missing = if_else(is.na(AGE), 1L, 0L),
    AGE_imputed = if_else(is.na(AGE), NA_real_, AGE),
    # leave imputation for model step (we will center and impute to mean with indicator)
    prop_ok = if_else(nr > 0, nriron / nr, NA_real_)  # undefined if nr==0
  )

glue <- function(...) paste0(...)

# Basic structure
nrow_dat <- nrow(dat)
colSums_na <- sapply(dat[,c("AGE","SEX","nriron","nr")], function(x) sum(is.na(x)))
list(n_rows = nrow_dat, na_counts = colSums_na)


In [ ]:
## 1.1) Descriptive statistics (no modeling)
# Summary for numeric variables and counts for factors
summ <- list(
  N = nrow(dat),
  AGE = list(mean = mean(dat$AGE, na.rm = TRUE), sd = sd(dat$AGE, na.rm = TRUE),
             min = min(dat$AGE, na.rm = TRUE), max = max(dat$AGE, na.rm = TRUE),
             n_missing = sum(is.na(dat$AGE))),
  SEX = list(counts = as.list(table(dat$SEX, useNA = "ifany"))),
  nriron = list(mean = mean(dat$nriron, na.rm = TRUE), sd = sd(dat$nriron, na.rm = TRUE),
                min = min(dat$nriron, na.rm = TRUE), max = max(dat$nriron, na.rm = TRUE)),
  nr = list(mean = mean(dat$nr, na.rm = TRUE), sd = sd(dat$nr, na.rm = TRUE),
            min = min(dat$nr, na.rm = TRUE), max = max(dat$nr, na.rm = TRUE)),
  prop_ok = list(mean = mean(dat$prop_ok, na.rm = TRUE), sd = sd(dat$prop_ok, na.rm = TRUE),
                 min = min(dat$prop_ok, na.rm = TRUE), max = max(dat$prop_ok, na.rm = TRUE),
                 n_missing = sum(is.na(dat$prop_ok)))
)
print(summ)

# Histograms: proportion (density of adequate iron stores), counts, number of measurements
p1 <- ggplot(dat, aes(x = prop_ok)) +
  geom_histogram(bins = 30, color = "white") +
  labs(x = "nriron / nr (proportion adequate)", y = "Number of patients",
       title = "Proportion of adequate iron stores")

ggsave("figs/p_hat_hist.png", p1, width = 7, height = 4.5, dpi = 150)

p2 <- ggplot(dat, aes(x = nriron)) +
  geom_histogram(binwidth = 1, boundary = -0.5, color = "white") +
  scale_x_continuous(breaks = 0:max(dat$nriron, na.rm = TRUE)) +
  labs(x = "Number of occasions with adequate iron stores (nriron)", y = "Number of patients",
       title = "Counts of adequate iron stores")

ggsave("figs/nriron_hist.png", p2, width = 7, height = 4.5, dpi = 150)

p3 <- ggplot(dat, aes(x = nr)) +
  geom_histogram(binwidth = 1, boundary = -0.5, color = "white") +
  scale_x_continuous(breaks = 0:max(dat$nr, na.rm = TRUE)) +
  labs(x = "Number of measurements (nr)", y = "Number of patients",
       title = "Distribution of number of measurements")

ggsave("figs/nr_hist.png", p3, width = 7, height = 4.5, dpi = 150)

list(figs = list("figs/p_hat_hist.png","figs/nriron_hist.png","figs/nr_hist.png"))


In [ ]:
## 2) Intercept-only Binomial GLM; Pearson dispersion
# Include all rows; if nr==0, Binomial contribution is undefined, so we set weights=0
# But assignment asks to include missing/NA; keep rows but ensure modeling is well-defined

dat_glm0 <- dat %>% mutate(
  y_success = nriron,
  y_failure = pmax(nr - nriron, 0L),
  w = if_else(nr > 0 & !is.na(y_success) & !is.na(y_failure), 1, 0)
)

m_bin0 <- glm(cbind(y_success, y_failure) ~ 1,
              family = binomial(), data = dat_glm0, weights = w)

pearson_r <- residuals(m_bin0, type = "pearson")
phi <- sum(pearson_r^2, na.rm = TRUE) / df.residual(m_bin0)

list(
  intercept_only = coef(summary(m_bin0)),
  pearson_dispersion = unname(phi)
)



In [ ]:
## 3) Binomial GLM with covariates (AGE, SEX); compare dispersion
# Handle AGE missing by mean-imputation plus missingness indicator to keep all rows
mu_age <- mean(dat$AGE, na.rm = TRUE)

dat_glmX <- dat %>% mutate(
  AGE_imp = if_else(is.na(AGE), mu_age, AGE),
  y_success = nriron,
  y_failure = pmax(nr - nriron, 0L),
  w = if_else(nr > 0 & !is.na(y_success) & !is.na(y_failure), 1, 0)
)

m_binX <- glm(cbind(y_success, y_failure) ~ AGE_imp + age_missing + SEX,
              family = binomial(), data = dat_glmX, weights = w)

phi_X <- {
  rp <- residuals(m_binX, type = "pearson")
  sum(rp^2, na.rm = TRUE) / df.residual(m_binX)
}

list(
  covariate_model = coef(summary(m_binX)),
  pearson_dispersion = unname(phi_X)
)


In [ ]:
## 4) CAMAN: Binomial mixture via VEM → EM and direct mixalg
# Use the counts (nriron) with trials (nr)
# Keep all rows; CAMAN naturally accommodates different ni. We keep rows with nr==0 as well.
# However, rows with nr==0 carry no binomial information; they will have zero likelihood information.

fit_vem <- NULL
fit_em_from_vem <- NULL
fit_npml <- NULL

if (requireNamespace("CAMAN", quietly = TRUE)) {
  # Phase 1: VEM (large grid)
  fit_vem <- CAMAN::mixalg.VEM(
    obs = "nriron",
    pop.at.risk = "nr",
    family = "binomial",
    data = dat,
    startk = 50,
    acc = 1e-8,
    numiter = 20000
  )
  
  # Phase 2: EM, seeding from VEM result
  fit_em_from_vem <- CAMAN::mixalg.EM(fit_vem)
  
  # Direct combined procedure
  fit_npml <- CAMAN::mixalg(
    obs = "nriron",
    pop.at.risk = "nr",
    family = "binomial",
    data = dat,
    startk = 50,
    acc = 1e-8,
    numiter = 50000,
    limit = 0.01
  )
  
  list(
    vem_summary = summary(fit_vem),
    em_from_vem = fit_em_from_vem,
    npml = summary(fit_npml)
  )
} else {
  message("CAMAN not available in this runtime. Mixture steps will be skipped. The code is ready and will run where CAMAN is installed.")
  list(vem_summary = NULL, em_from_vem = NULL, npml = NULL)
}


In [ ]:
## 5) Gradient function d(Ĝ, p) for Binomial mixture (NPMLE diagnostic)
# Use fitted NPML from fit_npml (only if CAMAN was available)

if (!is.null(fit_npml)) {
  # Extract support and weights
  p_hat <- fit_npml@t
  pi_hat <- fit_npml@p
  
  # Gradient function for Binomial mixture
  grad_binom <- function(p, y, n, pis, ps){
    denom <- vapply(seq_along(y), function(i) {
      sum(pis * dbinom(y[i], n[i], ps))
    }, numeric(1))
    # include all rows; for nr==0 the density is dbinom(0,0,p) == 1 for all p; safe
    mean(dbinom(y, n, p) / denom)
  }
  
  p_grid <- seq(1e-4, 1 - 1e-4, length.out = 800)
  d_vals <- sapply(p_grid, grad_binom,
                   y = dat$nriron, n = dat$nr,
                   pis = pi_hat, ps = p_hat)
  
  g_grad <- ggplot(data.frame(p = p_grid, d = d_vals), aes(p, d)) +
    geom_line() +
    geom_hline(yintercept = 1, linetype = 2) +
    geom_vline(xintercept = p_hat, colour = "red", alpha = .6) +
    labs(title = "Gradient d(Ĝ, p) – Binomial mixture NPMLE check",
         x = "p", y = "d(Ĝ, p)")
  
  ggsave("figs/gradient_binomial.png", g_grad, width = 7, height = 4.5, dpi = 150)
  
  "figs/gradient_binomial.png"
} else {
  message("Skipping gradient plot; CAMAN fit not available in this runtime.")
  NA_character_
}


In [ ]:
## 6) Overlay of mixture vs observed distribution of the outcome
# We overlay component-wise expected probability mass over the empirical distribution of y/n
# Because Binomial mixtures are about counts, we visualize the implied distribution of p (support t)
# against the histogram of observed proportions.

if (!is.null(fit_npml)) {
  # Density of proportions (histogram) + vertical lines at support points
  p_overlay <- ggplot(dat, aes(x = prop_ok)) +
    geom_histogram(aes(y = after_stat(density)), bins = 30, fill = "grey85", color = "white") +
    geom_vline(xintercept = p_hat, color = "steelblue", alpha = 0.7) +
    geom_point(data = data.frame(p = p_hat, w = pi_hat), aes(x = p, y = w),
               color = "red", size = 2) +
    labs(title = "Observed proportion distribution with mixing support and weights",
         x = "nriron / nr", y = "Density / Weights")
  
  ggsave("figs/overlay_mixture_NR3.png", p_overlay, width = 7, height = 4.5, dpi = 150)
  
  "figs/overlay_mixture_NR3.png"
} else {
  message("Skipping overlay plot; CAMAN fit not available in this runtime.")
  NA_character_
}


In [ ]:
## 7) Extract mixture details and concise interpretation helpers
mix_details <- if (!is.null(fit_npml)) {
  list(
    g_hat = fit_npml@num.k,
    support_p = fit_npml@t,
    weights = fit_npml@p,
    logLik = fit_npml@ll
  )
} else {
  NULL
}

# Pearson dispersion before and after covariates
phi0 <- phi
phi1 <- phi_X

list(mix_details = mix_details, pearson_dispersion_intercept = phi0,
     pearson_dispersion_covariates = phi1,
     dispersion_reduction = phi0 - phi1)


In [ ]:
## 8) Export key results for README update
results <- list(
  phi_intercept = unname(phi),
  phi_covariates = unname(phi_X),
  g_hat = if (!is.null(mix_details)) mix_details$g_hat else NA_integer_,
  support_p = if (!is.null(mix_details)) mix_details$support_p else NA,
  weights = if (!is.null(mix_details)) mix_details$weights else NA,
  logLik = if (!is.null(mix_details)) mix_details$logLik else NA
)
write_json(results, "analysis_results.json", pretty = TRUE)
"analysis_results.json"


In [ ]:
## 9) Export descriptive statistics
sex_counts <- as.list(table(dat$SEX, useNA = "ifany"))
descr <- list(
  N = nrow(dat),
  AGE = list(mean = mean(dat$AGE, na.rm = TRUE), sd = sd(dat$AGE, na.rm = TRUE),
             min = min(dat$AGE, na.rm = TRUE), max = max(dat$AGE, na.rm = TRUE),
             n_missing = sum(is.na(dat$AGE))),
  SEX = sex_counts,
  NR = list(mean = mean(dat$nr, na.rm = TRUE), sd = sd(dat$nr, na.rm = TRUE),
            min = min(dat$nr, na.rm = TRUE), max = max(dat$nr, na.rm = TRUE)),
  NRIRON = list(mean = mean(dat$nriron, na.rm = TRUE), sd = sd(dat$nriron, na.rm = TRUE),
                min = min(dat$nriron, na.rm = TRUE), max = max(dat$nriron, na.rm = TRUE)),
  PROP_OK = list(mean = mean(dat$prop_ok, na.rm = TRUE), sd = sd(dat$prop_ok, na.rm = TRUE),
                 min = min(dat$prop_ok, na.rm = TRUE), max = max(dat$prop_ok, na.rm = TRUE),
                 n_missing = sum(is.na(dat$prop_ok)))
)
write_json(descr, "descriptives.json", pretty = TRUE)
"descriptives.json"
